## SEBA example workflow 

This notebook covers the usage of most high level functions in SEBA. It consists of:
1. Reading and formatting behavioral annotation data from Boris/BehaView into text files with onset/offset timestamps synchronized to ephys time

2. Preparing a binary data structure (event is represented according to the recording where onset and offset is either read from data - for behaviors or provided otherwise by the user) that can easily be used for further modelling approaches like [CEBRA](https://github.com/AdaptiveMotorControlLab/CEBRA)
   
3. Analyzing the data by calculating firing rate, z-score for each event

4. Applying Wilcoxon rank sum test to establish neuron responsivness to each event

5. Using histology data from [HERBS](https://github.com/JingyiGF/HERBS) to match each neuron to a brain region it was recorded from

6. Create plots to explore the data:
   - Matrix of common neurons between the events
   - Matrix of N neurons responding to event vs structure
   - Heatmaps of neurons per recording or for all recordings for comparison of overall population activity to a specific event
   - PSTH plots to explore neural activity for each event
   - Paired heatmaps for comparison of overall population activity to two different events per recording or for all recordings
   - Paired PSTH plots to compare activity of specific neurons to 2 different events

### Import necessary packages

In [1]:
import os
from glob import glob
import seba
import pandas as pd

### Set paths to where the data is and where it should be saved

In [2]:
data_folder = glob(r"path_to_dir\*") # folder that contains folders with ephys recordings
histology_data = r"path_to_dir" # folder which contains all folders (per recording) generated by HERBS
save_results = r"path_to_dir" # folder in which the data structure and plots will be saved if dir doesn't exist it will be created

### Set behavior annotation data paths

In [ ]:
# NOTE: The way to get the path usually needs to be more specific especially when it comes to the file names for BORIS since csv is a common format
bvs = glob(r"path_to_dir\*.bvs")
boris = glob(r"path_to_dir\*.csv")

### Read and transform behavior annotations, append any external events

In [ ]:
# Example usage. NOTE: the file have to be matched to their recording directiories. It's easiest if the files are actually stored with the ephys recording itself
for file, save in zip(bvs, data_folder[:3]):
    seba.read_bvs(save, file, onsets_only=True)

for file, save in zip(boris, data_folder[3:]):
    seba.read_boris(save, file, onsets_only=True)

# NOTE: CS.txt specifies the name of the file that contains an external event. Name is assumed to be the same for each recording as it should represent the same type of an event
for dir in data_folder:
    seba.append_event_to_binary(dir, os.path.join(dir, r"CS.txt"), 20) 

### If not all recordings contain all events, specify the event names. 

<b>IMPORTANT:</b> Otherwise it's assumed that the first recording contains all possible events and names are derived from it!

In [ ]:
event_names = [
    "cs_onsets", 
    "freezing_experimental_onsets",
    "freezing_partner_onsets",
    "rearing_experimental_onsets",
    "rearing_partner_onsets",
    "relaxed_experimental_onsets",
    "relaxed_partner_onsets",
    "social_both_onsets",
]

### Read ephys data, perform analysis and create a data structure to store it. 

To check all `seba.structurize_data` attributes call `seba.structurize_data?`

If `calculate_responsive=True` it will perform Wilcoxon rank sum test and append information about neuron responsivness to the data structure.

In [ ]:
# Build and save data structure
data_obj = seba.structurize_data(
    data_folder=data_folder,
    event_names=event_names,
    save_path=save_results,
    pre_event=2,
    post_event=4,
    bin_size=0.02,
    calculate_responsive=True,
    p_bound=0.05,
    spike_threshold=5,
)

# Save information about responsivity to cluster_info_good.csv
seba.responsive_neurons2events(data_folder, data_obj)

### Read HERBS data and append information about brain regions to cluster_info_good.csv

In [ ]:
seba.get_brain_regions(data_folder, histology_folder, neuropixels_20=True | False)

seba.fix_wrong_shank_NP2(data_folder) # Take only the folders that use NP 2.0. Should be checked if new versions of HERBS are being used.

### Add brain region information to data_obj

In [ ]:
seba.add_brain_regions(data_folder, data_obj)

### If running the notebook again and ephys_data.pickle already exist read it with the cell below

In [3]:
#data_obj = pd.read_pickle(r"path_to_ephys_data.pickle")

### Define pairs of behaviors you want to explore. Those will be used for paired plots

In [ ]:

behavioral_pairs = [["freezing_experimental_onsets", "freezing_partner_onsets"],
                    ["freezing_experimental_onsets", "relaxed_experimental_onsets"],
                    ["rearing_experimental_onsets", "rearing_partner_onsets"],
                    ]

### Run plotting

`per_recording`, `z_score`, `responsive_only` are main attributes users can modify where:
1. `per_recording=True` means that data is plotted per recording, otherwise it's plotted for all recordings combined
2. `z_score=True` means that z-scored data is plotted, otherwise firing rate is used
3. `responsive_only=True` means that only responsive neurons are taken into account when creating plots. Otherwise activity of all neurons is plotted.
4. `y_limit` specified the y axis limits for line plots (PSTH)
5. `percent_all_neurons=True` means that the matrix will show a percentage of all neurons for recording/all recordings (depends on `per_recording`) instead of the number of neuorns

In [ ]:
seba.plot_common_nrns_matrix(data_obj, save_results, per_recording=False, percent_all_neurons=True)
seba.plot_common_nrns_matrix(data_obj, save_results, per_recording=True, percent_all_neurons=True)
for pair in behavioral_pairs:
    seba.plot_heatmaps_paired(data_obj, pair, save_path=save_results, per_recording=False, responsive_only=True, colormap = "inferno", z_score=True)
    seba.plot_lin_reg_scatter(data_obj, pair, save_path=save_results, per_recording=False, responsive_only=True)
    seba.plot_psths_paired(data_obj, pair, save_path=save_results, responsive_only=True, z_score=True, ylimit= [-2, 4])

seba.plot_psths(data_obj, save_results, ylimit= [-2, 4], responsive_only=True)
seba.plot_heatmaps(data_obj, save_results, responsive_only=True)
seba.neurons_per_structure(data_folder, data_obj, save_results)
seba.plot_neurons_per_event_structure(data_folder, data_obj, save_results, per_recording=False)